In [55]:
from datasets import load_dataset
bookcorpus_dataset = load_dataset('bookcorpus')

# 查看数据集中包含的split。常见的split有'train', 'test', 'validation'。
print(bookcorpus_dataset.keys())

# 查看某个split（如'train'）的数据大小
print(len(bookcorpus_dataset['train'])*0.01)

# 查看数据集的一些样例
print(bookcorpus_dataset['train'][0])


Found cached dataset bookcorpus (/home/dtdysh/.cache/huggingface/datasets/bookcorpus/plain_text/1.0.0/eddee3cae1cc263a431aa98207d4d27fd8a73b0a9742f692af0e6c65afa4d75f)
100%|██████████| 1/1 [00:17<00:00, 17.63s/it]


dict_keys(['train'])
740042.28
{'text': 'usually , he would be tearing around the living room , playing with his toys .'}


In [57]:
print(len(bookcorpus_dataset['train'])*0.001)


74004.228


In [68]:
import uuid
import json
from tqdm import tqdm

def convert_to_csqa(example):
    # 获取当前text的最后一个词作为正确答案
    last_word = example["text"].split()[-1]

    # 构建CSQA格式的数据
    csqa_example = {
        "id": str(uuid.uuid4()),  # 使用uuid4生成随机id
        "question": {
            "choices": [{"label": "A", "text": last_word}],  # 只有一个选项，即正确答案
            "stem": example["text"]  # 原文本作为问题描述
        },
        "answerKey": "A"  # 因为只有一个选项，所以答案键始终为A
    }

    return csqa_example

dataset_size = 225000
train_size = int(dataset_size * 0.8)  # 80% for training
test_size = int(dataset_size * 0.1)  # 10% for testing
dev_size = dataset_size - train_size - test_size  # 10% for dev

# IDs for training set
ids_train = []

# Write the training data and testing data
with open('../data/bookcorpus/train_rand_split.jsonl', 'w') as f:
    for i in tqdm(range(train_size + test_size), desc="Writing train and test data"):
        csqa_example = convert_to_csqa(bookcorpus_dataset['train'][i])
        if i < train_size:  # Only save the id for real training set
            ids_train.append(csqa_example["id"])  # Save ID
        f.write(json.dumps(csqa_example) + '\n')  # Write each example in JSON format

# Write the dev data
with open('../data/bookcorpus/dev_rand_split.jsonl', 'w') as f:
    for i in tqdm(range(train_size + test_size, dataset_size), desc="Writing dev data"):
        csqa_example = convert_to_csqa(bookcorpus_dataset['train'][i])
        f.write(json.dumps(csqa_example) + '\n')  # Write each example in JSON format

# Write IDs for training set
with open('../data/bookcorpus/inhouse_split_qids.txt', 'w') as f:
    for id in ids_train:
        f.write(f"{id}\n")

print("done")


Writing dev data: 100%|██████████| 22500/22500 [00:01<00:00, 11743.55it/s]

done


### Then create or copy a test_rand_split_not_answers.jsonl as test set. But we will only use inhouse test set